***Vorlesung 'Syntax natürlicher Sprachen', WS 2020/21***

---
# Übung 4

In [1]:
from exercises_4 import *
from questions import aufgabe
import nltk
import sys

---

## Aufgabe 1: Eine erste (Phrasenstruktur-)Grammatik

#### Werfen Sie einen Blick auf die folgende sehr einfache kontextfreie Grammatik und erklären Sie deren Funktionsweise.

In [14]:
grammar1 = """
    S -> NP VP
    NP -> DET N
    VP -> V NP NP
    
    V -> "gibt" | "schenkt"
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
"""

---
#### Sammeln Sie Sätze, die als grammatisch erkannt werden sollten, am besten in einer Liste.

In [3]:
test_sentences = [
    "der Mann gibt der Frau das Buch"
]

---
#### Die folgende Funktion kann Ihnen helfen, eine Reihe von Sätzen zu analysieren.

In [5]:
def test_grammar(grammar, sentences):
    cfg = nltk.CFG.fromstring(grammar)
    rd_parser = nltk.RecursiveDescentParser(cfg)
    
    for i, sent in enumerate(sentences, 1):
        print("Satz {}: {}".format(i, sent))
        results = rd_parser.parse(sent.split())
        analyzed = False
        for tree in results:
            tree.pretty_print(unicodelines=True)
            analyzed = True
        if not analyzed:
            print("Keine Analyse möglich", file=sys.stderr)

---
#### Führen Sie den Test jetzt für `grammar1` aus!

In [15]:
test_grammar(grammar1, test_sentences)

Satz 1: der Mann gibt der Frau das Buch
                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch



---
## Aufgabe 2: Ein- und zweistellige Verben

#### Bis jetzt akzeptiert die Grammatik in `grammar1` lediglich dreistellige Verben. Erweitern Sie sie so, dass auch Verben mit weniger als zwei Objekten korrekte Verbalphrasen bilden können.

#### Folgende Sätze sollten akzeptiert werden:

In [16]:
test_sentences.extend([
    "der Mann schläft",
    "das Buch gefällt der Frau",
    "die Frau kennt das Buch"
])

In [36]:
grammar2 = """
    S -> NP VP
    NP -> DET N
#ditransitive VP:
    VP -> V NP NP    
#transitive VP:
    VP -> V NP
#intransitive VP:
    VP -> V
   
    V -> "gibt" | "schenkt" | "schläft"  | "gefällt"  | "kennt"
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
"""

In [37]:
test_grammar(grammar2, test_sentences)

Satz 1: der Mann gibt der Frau das Buch
                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch

Satz 2: der Mann schläft
         S          
     ┌───┴──────┐    
     NP         VP  
 ┌───┴───┐      │    
DET      N      V   
 │       │      │    
der     Mann schläft

Satz 3: das Buch gefällt der Frau
                S                
     ┌──────────┴─────┐           
     │                VP         
     │          ┌─────┴───┐       
     NP         │         NP     
 ┌───┴───┐      │     ┌───┴───┐   
DET      N      V    DET      N  
 │       │      │     │       │   
das     Buch gefällt der     Frau

Satz 4: die Frau kennt das Buch
      

---

## Aufgabe 3: Beliebig lange Phrasen

#### Erweitern Sie die Grammatik nun derart, dass Nominalphrasen auch Adjektive enthalten dürfen – und zwar beliebig viele.

#### Beispiele:

In [38]:
test_sentences.extend([
    "die kluge schöne Frau kennt das Buch",
    "der schöne kluge Mann gibt der Frau das dicke Buch",
    "das dicke schöne kluge Buch schläft"
])

In [54]:
grammar3 = """
    S -> NP VP
    
#Spezifizierer-Regel:    
    NP -> DET NOM
#ADJUNKT-Regel:    
    NOM -> ADJ NOM
#KOMPLEMENT-Regel (leer):    
    NOM -> N
    
    VP -> V NP NP | V NP | V
    
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    V -> "gibt" | "schenkt" | "schläft"  | "gefällt"  | "kennt"
    ADJ -> "kluge" | "schöne" | "dicke"
"""

In [ ]:
##NEGATIV-BEISPIEL:
test_sentences.extend([
    "das dicke schöne kluge das Buch schläft"
])

In [55]:
test_grammar(grammar3, test_sentences)

Satz 1: der Mann gibt der Frau das Buch
                   S                       
     ┌─────────────┴───────┐                
     │                     VP              
     │        ┌────────┬───┴────────┐       
     NP       │        NP           NP     
 ┌───┴───┐    │    ┌───┴───┐    ┌───┴───┐   
 │      NOM   │    │      NOM   │      NOM 
 │       │    │    │       │    │       │   
DET      N    V   DET      N   DET      N  
 │       │    │    │       │    │       │   
der     Mann gibt der     Frau das     Buch

Satz 2: der Mann schläft
         S          
     ┌───┴──────┐    
     NP         │   
 ┌───┴───┐      │    
 │      NOM     VP  
 │       │      │    
DET      N      V   
 │       │      │    
der     Mann schläft

Satz 3: das Buch gefällt der Frau
                S                
     ┌──────────┴─────┐           
     │                VP         
     │          ┌─────┴───┐       
     NP         │         NP     
 ┌───┴───┐      │     ┌───┴───┐   
 │      NO

Keine Analyse möglich


---
# Hausaufgaben

---
## Aufgabe 4: Eigennamen

#### Eigennamen können auch ohne Artikel eine Nominalphrase bilden. Erweitern Sie die Grammatik entsprechend.

#### Folgende Sätze sollten korrekt analysiert werden:

In [ ]:
test_sentences.extend([
    "der Mann kennt Chomsky",
    "Marie gibt Fritz das Buch"
])

#### Stellen Sie außerdem sicher, dass folgende Sätze KEINE Analyse liefern!

In [ ]:
negative_examples = [
    "Mann gibt Frau Buch",
    "Mann schläft"
]

In [ ]:
grammar4 = """
    S -> NP VP
    NP -> DET N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP
    V -> "gibt" | "schenkt"
"""

In [ ]:
test_grammar(grammar4, test_sentences)

---
## Aufgabe 5: Präpositionalphrasen

#### Erweitern Sie die Grammatik nun derart, dass sowohl Nominalphrasen als auch Verbalphrasen durch Präpositionalphrasen modifiziert werden können. Erkennen Sie die Ambiguität von Übungsblatt 2 wieder?

#### Folgende Sätze sollten akzeptiert werden:

In [ ]:
test_sentences.extend([
    "der Mann schläft neben dem Buch",
    "die Frau kennt das dicke Buch über Chomsky",
    "die Frau schenkt dem Mann das Buch auf dem Tisch"
])

#### Beachten Sie, dass die Form des Artikels *dem* und das Nomen *Tisch* auch noch als lexikalische Regeln ergänzt werden müssen. 

#### Stellen Sie zudem sicher, dass für den 2. und 3. Satz zwei Analysen generiert werden.

#### Achten Sie außerdem darauf, dass Ihre Erweiterung nicht die Analyse zuvor akzeptierter grammatischer Sätze verhindert!

In [ ]:
grammar5 = """
    S -> NP VP
    NP -> DET N
    DET -> "der" | "die" | "das"
    N -> "Mann" | "Frau" | "Buch"
    VP -> V NP NP
    V -> "gibt" | "schenkt"
"""

In [ ]:
test_grammar(grammar5, test_sentences)

---
## Aufgabe 6: Fragen zu NLTK-Kapitel 8.3

#### Lesen Sie das NLTK-Teilkapitel 8.3 (’Context Free Grammar’): http://www.nltk.org/book/ch08.html.

#### Beantworten Sie insbesondere folgende Fragen:

---
### Aufgabe 6 a:

In [6]:
aufgabe(blatt4_6a)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Wie ist im NLTK vorzugehen, wenn ein (wohlgef…

---
### Aufgabe 6 b:

In [7]:
aufgabe(blatt4_6b)

OpenTextfield(children=(HTML(value='<h4 style="font-size:14px;">Wie können im NLTK die Regeln einer Grammatik …

---
### Aufgabe 6 c:

#### Geben Sie 2 NP-Phrasenstrukturregeln mit indirekter Rekursion an und beantworten Sie danach die folgende Frage!

In [8]:
aufgabe(blatt4_6c)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele zeigen direkte Rekursion?</h…

---
### Aufgabe 6 d:

#### Geben Sie ein Beispiel für eine links- sowie eine rechtsrekursive NP-Phrasenstrukturregel an.

#### Beantworten Sie auch die folgenden Fragen:

In [9]:
aufgabe(blatt4_6d)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele für Phrasenstrukturregeln si…

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Welche Beispiele sind rechtsrekursiv?</h4>'),…